<a href="https://colab.research.google.com/github/rupalidawkoregithub/Book-Recommendation-System/blob/main/Book_Recommendation_System_Unsupervised_ML_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    -  Book Recommendation System



##### **Project Type**    - Unsupervised
##### **Contribution**    - Individual Notebook
##### **Team Member 1 -** Rupali Dawkore


# **Project Summary -**


##**Problem Statement:**

The rise of internet services such as Youtube, Amazon, and Netflix has resulted in an increase in the popularity of recommender systems. Recommender systems are algorithms designed to suggest relevant items to users based on their preferences. They have become essential in various industries, including e-commerce, online advertising, and streaming services. In this project, we will develop a book recommendation system for users based on the Book-Crossing dataset. We will use unsupervised machine learning techniques such as exploratory data analysis, content-based filtering.
##**Book-Crossing Dataset:**

The Book-Crossing dataset contains three files: Users, Books, and Ratings. The Users file contains demographic information about users, including their location and age. The Books file provides information about books, including their ISBN, author, title, and publication year. The Ratings file contains information about the ratings given by users to books. Ratings range from 1 to 10, with higher values indicating higher appreciation. Some ratings are implicit and expressed by 0.5.

##**Exploratory Data Analysis (EDA):**

Before developing our recommendation models, we will perform EDA to understand the characteristics of the dataset. EDA helps us to identify missing values, outliers, and correlations between variables. It also provides insights into the distribution of data, which can inform our modeling decisions.

##**Content-Based Filtering:**

Our ML model is based on content-based filtering.The ML model 1 is based on content-based filtering. It recommends books to users based on their past preferences. The model first cleans and preprocesses the dataset by removing unwanted columns and duplicating rows. It then vectorizes the text data by tokenizing and converting them into numerical data using TfidfVectorizer. After vectorizing the text data, it creates a cosine similarity matrix that measures the similarity between each book based on their textual features. Finally, the model recommends top-n similar books to a given book based on their cosine similarity score.

Overall, ML models are designed to recommend books to users based on their preferences. Model 1 is based on the textual features of the books.

##**Conclusion:**

The development of recommender systems has become increasingly important in various industries, including e-commerce, online advertising, and streaming services. the book recommendation system developed using the Book-Crossing dataset has ML models based on unsupervised learning techniques: content-based filtering. These models use different approaches to recommend books to users based on their preferences. The content-based filtering approach recommends books based on the textual features of the books.

# **GitHub Link -**

Rupali Dawkore : https://github.com/rupalidawkoregithub/Book-Recommendation-System

# **Problem Statement**


During the last few decades, with the rise of Youtube, Amazon, Netflix, and many other such web services, recommender systems have taken more and more place in our lives. From e-commerce (suggest to buyers articles that could interest them) to online advertisement (suggest to users the right contents, matching their preferences), recommender systems are today unavoidable in our daily online journeys.In a very general way, recommender systems are algorithms aimed at suggesting relevant items to users (items being movies to watch, text to read, products to buy, or anything else depending on industries).

Recommender systems are really critical in some industries as they can generate a huge amount of income when they are efficient or also be a way to stand out significantly from competitors.

The main objective is to create a book recommendation system for users.

Content
The Book-Crossing dataset comprises 3 files.

● Users

Contains the users. Note that user IDs (User-ID) have been anonymized and map to integers. Demographic data is provided (Location, Age) if available. Otherwise, these fields contain NULL values.

● Books

Books are identified by their respective ISBN. Invalid ISBNs have already been removed from the dataset. Moreover, some content-based information is given (Book-Title, Book-Author, Year-Of-Publication, Publisher), obtained from Amazon Web Services. Note that in the case of several authors, only the first is provided. URLs linking to cover images are also given, appearing in three different flavors (Image-URL-S, Image-URL-M, Image-URL-L), i.e., small, medium, large. These URLs point to the Amazon website.

● Ratings

Contains the book rating information. Ratings (Book-Rating) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required. 
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits. 
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule. 

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Importing All Neccessary Libraries
# importing regular expression library
import re
# importing pickle library for object serialization
import pickle
# importing operator library for efficient operators
import operator
# importing pandas library for data manipulation
import pandas as pd
# Importing NumPy library for numerical computations and array manipulations
import numpy as np 
# importing matplotlib library for data visualization
import matplotlib.pyplot as plt
# importing seaborn library for statistical graphics
import seaborn as sns
# importing missingo library which helps us to visualize the missing values
import missingno as msno
# This is to supress the warning messages (if any) generated in our code
import warnings
warnings.filterwarnings(action="ignore")
# importing scipy library for scientific computing and technical computing
import scipy
# importing math library for mathematical operations
import math
# importing random library for generating random numbers
import random
# importing sklearn library for machine learning algorithms and tools
import sklearn
# importing stopwords from NLTK corpus to remove stopwords from text data
from nltk.corpus import stopwords
# importing train_test_split function from sklearn library for splitting dataset
from sklearn.model_selection import train_test_split 
# importing CountVectorizer from sklearn library for text preprocessing
from sklearn.feature_extraction.text import CountVectorizer
# importing normalize function from sklearn library for normalizing the data
from sklearn.preprocessing import normalize
# importing svds function from scipy.sparse.linalg library for singular value decomposition
from scipy.sparse.linalg import svds
# Importing the Counter class from the collections module
from collections import Counter
# Importing the compressed sparse row matrix class from scipy.sparse module    
from scipy.sparse import csr_matrix 
# Importing the is_numeric_dtype function from the pandas.api.types module  
from pandas.api.types import is_numeric_dtype
# Importing the NearestNeighbors class from the sklearn.neighbors module
from sklearn.neighbors import NearestNeighbors
# Importing the DictVectorizer class from the sklearn.feature_extraction module   
from sklearn.feature_extraction import DictVectorizer
# Importing the cosine_similarity function from the sklearn.metrics.pairwise module   
from sklearn.metrics.pairwise import cosine_similarity
# Importing the TfidfVectorizer class from the sklearn.feature_extraction.text module   
from sklearn.feature_extraction.text import TfidfVectorizer
# Importing the mstats module from the scipy.stats module   
from scipy.stats import mstats
# Importing the trim_mean function from the scipy.stats module   
from scipy.stats import trim_mean 


### Dataset Loading

In [ ]:
# Mount google drive for access the dataset of Book Recommendation System
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading datasets of Book Recommendation Systems 
books_df = pd.read_csv("/content/drive/MyDrive/Book Recommendation /Books.csv")     # Read books csv file
users_df = pd.read_csv("/content/drive/MyDrive/Book Recommendation /Users.csv")     # Read users csv file
ratings_df = pd.read_csv("/content/drive/MyDrive/Book Recommendation /Ratings.csv") # Read users csv file

In [ ]:
# Checking shape of each dataframe 
print("Books Data:", books_df.shape)        # Checked shape of books dataframe
print("Books-ratings :", ratings_df.shape)  # Checked shape of ratings dataframe
print("Users Data:", users_df.shape)        # Checked shape of users dataframe


###**Books Dataset**

##### Dataset First View

In [ ]:
# Dataset First Look
books_df.info() 

In [ ]:
# This code sets the maximum column width to be unlimited.
pd.set_option('display.max_colwidth', -1) 

#####Dataset Rows & Columns count

In [ ]:
# Get the number of rows and columns
rows, columns = books_df.shape 

In [ ]:
# Print the number of rows and columns
print("Number of rows: ", rows)
print("Number of columns: ", columns) 

In [ ]:
# checking all columns from books dataframe and looking forward the first 5 rows
print("Columns: ", list(books_df.columns))
books_df.head() 

#####Dataset Information

In [ ]:
# Checking book dataset information
books_df.info() 

#####Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print(f"Number of duplicated rows in books dataset: {books_df.duplicated().sum()}") 

There is No Duplicate Values in Books Dataframe.

#####Dropping Columns

In [ ]:
# Dropping Unnecessary URL columns
books_df.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
books_df.head() 

#####Missing Values/Null Values

In [ ]:
# Making function to check missing or null values of dataframes
def missing_values(df): 
    mis_val=df.isnull().sum()
    mis_val_percent=round(df.isnull().mean().mul(100),2)
    mz_table=pd.concat([mis_val,mis_val_percent],axis=1)
    mz_table=mz_table.rename(
    columns={df.index.name:'col_name',0:'Missing Values',1:'% of Total Values'})
    mz_table['Data_type']=df.dtypes
    mz_table=mz_table.sort_values('% of Total Values',ascending=False)
    return mz_table.reset_index()

In [ ]:
# Checking missing values for books dataframe
missing_values(books_df) 

In [ ]:
# Visualizing the missing values
msno.bar(books_df,figsize=(10,5), color="tab:cyan") 

We have missing value in Book-Author & Publisher.   

In [ ]:
# Checking Index of null values 
books_df.loc[books_df['Book-Author'].isnull(),:] 

In [ ]:
# Filling the missing value of book author with other
books_df.at[187689 ,'Book-Author'] = 'Others' 

In [ ]:
# Checking Index of null values 
books_df.loc[books_df['Publisher'].isnull(),:] 

In [ ]:
# Filling the missing value of publisher with other
books_df.at[128890 ,'Publisher'] = 'Others'
books_df.at[129037 ,'Publisher'] = 'Others' 

#### ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
books_df.columns 

In [ ]:
# Dataset Describe 
books_df.describe()

##### Variables Description 

 **1.  ISBN :**  This column represents the unique International Standard Book Number for each book in the dataframe. Each book has a distinct ISBN.

 **2.  Book-Title   :**  This column represents the title of each book. It is a categorical variable that can take on one of several possible values, such as "Classical Mythology", "Clara Callan"

 **3.  Book-Author       :**  This column represents the author of each book. It is a categorical variable.

 **4.Year-Of-Publication :** This column represents the year in which each book was published. It is a discrete numerical variable that can take on integer values greater than or equal to 0, such as 2002, 2001, 1991, 1999, and so on.

 **5.Publisher :**  This column represents the publisher of each book. It is a categorical variable that can take on one of several possible values.

 **6.Image-URL-S :**  This column represents the URL for a small image of the book cover. It is a categorical variable that contains the URL for a thumbnail-sized image of the book cover.

 **7.Image-URL-M:**  This column represents the URL for a medium image of the book cover. It is a categorical variable that contains the URL for a medium-sized image of the book cover.

 **8.Image-URL-L:**  This column represents the URL for a large image of the book cover. It is a categorical variable that contains the URL for a full-sized image of the book cover.

##### Check Unique Values for each variable.

In [ ]:
# looking for columns in Books dataset
books_df.columns 

In [ ]:
# Function for printing the Unique Values of Each column in dataframe
def get_all_unique_values(df):
    for col in df.columns:
        print(f"Unique values in column '{col}':")
        print(df[col].unique()) 
        print()

In [ ]:
# Get and print all unique values
get_all_unique_values(books_df) 

#### 3. ***Data Wrangling***

##### Data Wrangling Code

In [ ]:
# Checking unique values for column Year-of-publication
books_df['Year-Of-Publication'].unique() 

In [ ]:
# Checking the values of DK Publishing Inc
books_df.loc[books_df['Year-Of-Publication'] == 'DK Publishing Inc',:] 

In [ ]:
# Checking the values of Gallimard
books_df.loc[books_df['Year-Of-Publication'] == 'Gallimard',:] 

**As it can be seen from above that there are some incorrect entries in Year-Of-Publication field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as Year-Of-Publication in dataset due to some errors in csv file.**

In [ ]:
#From above, it is seen that bookAuthor is incorrectly loaded with Year-Of-Publication, hence making required corrections
#ISBN '078946697X'
books_df.at[209538 ,'Publisher'] = 'DK Publishing Inc'
books_df.at[209538 ,'Year-Of-Publication'] = 2000
books_df.at[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
books_df.at[209538 ,'Book-Author'] = 'Michael Teitelbaum' 

In [ ]:
#ISBN '0789466953'
books_df.at[221678 ,'Publisher'] = 'DK Publishing Inc'
books_df.at[221678 ,'Year-Of-Publication'] = 2000
books_df.at[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books_df.at[209538 ,'Book-Author'] = 'James Buckley' 

In [ ]:
#ISBN '2070426769'
books_df.at[220731 ,'Publisher'] = 'Gallimard'
books_df.at[220731 ,'Year-Of-Publication'] = 2003
books_df.at[209538 ,'Book-Title'] = 'Peuple du ciel - Suivi de Les bergers '
books_df.at[209538 ,'Book-Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

In [ ]:
# convert "Fee" from String to int
books_df= books_df.astype({'Year-Of-Publication':'int'})
print(books_df.dtypes)

In [ ]:
books_df["Year-Of-Publication"].shape

In [ ]:
books_df['Year-Of-Publication'].unique() 

In [ ]:
sorted_years = sorted(books_df['Year-Of-Publication'].unique())
print(sorted_years)

**The value 0 for Year-Of_Publication is invalid and as this dataset was published in 2004, We have assumed that the years after 2004 to be invalid and setting invalid years as NaN**

In [ ]:
# Replace values greater than 2006 or equal to 0 with NaN
# books_df.loc[(books_df['Year-Of-Publication'] > 2004) | (books_df['Year-Of-Publication'] == 0),'Year-Of-Publication'] = np.NAN
# # Fill NaN values with the median value of the column
# books_df['Year-Of-Publication'].fillna(round(books_df['Year-Of-Publication'].median()), inplace=True) 

In [ ]:
# Convert invalid values to NaN
mask = (books_df['Year-Of-Publication'] > 2004) | (books_df['Year-Of-Publication'] == 0)
books_df.loc[mask, 'Year-Of-Publication'] = np.nan
# Replace NaN values with median of remaining valid values
median_year = books_df['Year-Of-Publication'].median()
books_df['Year-Of-Publication'].fillna(median_year, inplace=True) 

The 'Year-Of-Publication' column in the dataset is of integer type, with values ranging from 0 to 2050. However, as the dataset was created in 2004, it is assumed that all years after 2004 are invalid, allowing for a margin of two years in case the dataset was updated. Any invalid entries in this column, including those with a value of 0, will be converted to NaNs using a boolean mask. Afterward, the remaining valid values in the column will be used to calculate the median year. Finally, all NaN values in the column will be replaced with the calculated median year. This process ensures that the data is clean and usable for analysis.

In [ ]:
# Rechecking unique values for year of publication
books_df['Year-Of-Publication'].unique()   

###**Users Dataset**

#####Users Dataset First Look

In [ ]:
# Users Dataset First Look
users_df.head()     

In [ ]:
# Checking missing values for users dataframe
missing_values(users_df) 

##### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
row,columns=users_df.shape
# Print the size of row & column of users dataset 
print("Rows of Users Df",row) 
print("Columns of Users Df",columns)    

##### Dataset Information

In [ ]:
# Checking dataset information 
users_df.info()    

* This dataset consists of 3 features with 278858 entries with 'Age' column having Null Values.

* Age Column have lots of Null Values

##### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print(f"Number of duplicated rows in users dataset: {users_df.duplicated().sum()}")   

There is No Duplicates found.

#### ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
users_df.columns   

In [ ]:
# Dataset Describe
users_df.describe()  

### Variables Description 

 **1.  User-ID   :**  This column represents the unique identifier for each user in the dataframe. Each user has a distinct User-ID.

 **2.  Location  :**  This column represents the location of each user. It is a categorical variable that can take on one of several possible values, such as "nyc, new york, usa", "stockton, california, usa."

 **3.  Age       :** This column represents the age of each user in years. It is a continuous numerical variable that can take any value greater than or equal to 0.

##### Check Unique Values for each variable.

In [ ]:
# looking for columns in users dataset
users_df.columns  

In [ ]:
# Get and print all unique values
get_all_unique_values(users_df) 

##### Missing Values/Null Values

In [ ]:
# Checking null values
missing_values(users_df)  

**Age have around 39% missing values.**

In [ ]:
# Visualizing the missing values
msno.bar(users_df,figsize=(10,5), color="tab:blue") 

In [ ]:
# Check for all values present in Age column
users_df['Age'].unique()   

In [ ]:
# distribution of Age column
plt.figure(figsize=(8,4)) 
sns.distplot(users_df['Age']); 

* The feature Age is Rightly skewed. Replacing Null values with Median value

Upon checking the unique values, userID looks correct. However, Age column has a NaN and some very high values. In my view ages below 5 and above 90 do not make much sense, and hence, these are being replaced with NaNs. All the NaNs are then replaced with median value of Age, and its data type is set as int.

In [ ]:
# Sorting the unique values of age columns
print(sorted(list(users_df['Age'].unique())))   

As can be seen from the values that there are people who are 0 years old and also we have people who are 244 years old which is for sure an error so we will keep the age group only from 5 years old to 100 years old and for the rest we will replace them with median of the age.

In [ ]:
# Replacing the less than 5 and more than 85 values with mean of the age   
# users_df.loc[(users_df['Age'] < 5) | (users_df['Age'] > 100),'Age'] = np.nan
# users_df['Age'].fillna((users_df['Age'].median()), inplace=True)    

In [ ]:
# Convert invalid values to NaN
masking= (users_df['Age'] > 100) | (users_df['Age'] < 5)
users_df.loc[masking, 'Age'] = np.nan

# Replace NaN values with median of remaining valid values
median_age = users_df['Age'].median()
users_df['Age'].fillna(median_age, inplace=True)

In [ ]:
# Rechecking unique values for age columns
print(sorted(users_df['Age'].unique())) 

In [ ]:
# Rehecking the null values
missing_values(users_df)  

* No Null values are present in Age column.

#### 3. ***Data Wrangling***

##### Data Wrangling Code

Let's check for outliers in age column

In [ ]:
# Checking outliers for age
sns.boxplot(y='Age', data=users_df)
plt.title('Find outlier data in Age column') 

There no more outliers in age column.

In [ ]:
# Checking location with unique 
users_df.Location.unique() 

In [ ]:
# Checking number of unique values for location 
users_df.Location.nunique() 

**57339 unique Value it's really hard to understand,
So lets create column Country.**

In [ ]:
# def extract_country(users_df):
#     users_df['Country'] = users_df['Location'].str.extract(r',\s?(\w+\s*\w*)\"*$')
#     return users_df   

In [ ]:
# Making function to extract country from location in users dataframe
import re

def extract_country(location):
    pattern = r',\s?(\w+\s?\w*)\"*$'
    match = re.search(pattern, location)
    if match:
        return match.group(1)
    else:
        return None

users_df['Country'] = users_df['Location'].apply(extract_country)

In [ ]:
#extract_country(users_df) 

In [ ]:
# Checking unique numbers for country
users_df.Country.nunique()  

In [ ]:
#drop location column
users_df.drop('Location',axis=1,inplace=True)  

In [ ]:
# Checking head of users df
users_df.head(2)  

In [ ]:
# Rechecking missing values of users dataframe
missing_values(users_df)  

In [ ]:
#users_df['Country']=users_df['Country'].astype('str') 

In [ ]:
# Printing country
print(users_df['Country'])  

In [ ]:
# Printing first 10 country
print(users_df['Country'][:10])  

In [ ]:
# Let's Replace some Misspelt Countries 
users_df['Country'].replace(['','01776','02458','19104','23232','30064','85021','87510','alachua','america','austria',
                             'autralia','cananda','geermany','germay','italia','united kindgonm','united sates','united staes',
                             'united state','united states','us','urugua','indiai','canada eh','le canada','nan'],
                           ['others','usa','usa','usa','usa','usa','usa','usa','usa','usa','australia','australia',
                            'canada','germany','germany','italy','united kingdom','usa','usa','usa','usa','usa',
                            'uruguay','india','canada','canada','others'],inplace=True)  

In [ ]:
users_df['Country'].fillna('other',inplace=True)  

In [ ]:
users_df.head() 

In [ ]:
# Rechecking missing values
missing_values(users_df)  

In [ ]:
users_df.shape 

###**Ratings Dataset**

#####Ratings Dataset First Look

In [ ]:
#first look for rating dataset
ratings_df.head()   

In [ ]:
ratings_df.shape 

##### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
row,col=ratings_df.shape
print("Row:",row)
print("col:",col)   

In [ ]:
# Checking all columns from ratings dataframe
print("Columns: ", list(ratings_df.columns))  

##### Dataset Information

In [ ]:
# Dataset First Look
ratings_df.info()  

* There is No Null Values in the above dataset

##### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print(f"Number of duplicated rows in ratings dataset: {ratings_df.duplicated().sum()}")  

There is No Duplicates found.

#### ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
ratings_df.columns  

In [ ]:
# Dataset Describe
ratings_df.describe()  

### Variables Description 

 **1.  User-ID           :**   This column represents the unique identifier for each user in the dataframe. Each row represents a rating given by a user for a particular book, and the User-ID column specifies which user gave the rating.

 **2.  ISBN              :**  This column represents the unique identifier for each book in the dataframe. Each row represents a rating given by a user for a particular book, and the ISBN column specifies which book was rated.

 **3.  Book-Rating       :**  This column represents the rating given by a user for a particular book. It is a numerical variable that can take on integer values between 0 and 10.
 

##### Check Unique Values for each variable.

In [ ]:
# Get and print all unique values 
get_all_unique_values(ratings_df) 

##### Missing Values/Null Values

In [ ]:
# Checking for null values of ratings dataframe
missing_values(ratings_df)  

In [ ]:
# Visualizing the missing values
msno.bar(ratings_df,figsize=(10,5), color="tab:green")  

#### 3. ***Data Wrangling***

##### Data Wrangling Code

In [ ]:
# Looking for first row
ratings_df.head(1)  

**Ratings dataset should have books only which exist in our books dataset.**

In [ ]:
# Keeping only the records of ratings for the books and users whose data is there in the respective users and books csv file
#ratings_df = ratings_df[ratings_df['ISBN'].isin(books_df['ISBN'])]         # Filter ratings to only include ISBN present in books_df 
#ratings_df = ratings_df[ratings_df['User-ID'].isin(users_df['User-ID'])] # Filter ratings to only include users present in users_df


In [ ]:
# Filter ratings to only include ISBN present in books_df
ratings_df = ratings_df[ratings_df['ISBN'].isin(books_df['ISBN'])]

# Filter ratings to only include users present in users_df
ratings_df = ratings_df[ratings_df['User-ID'].isin(users_df['User-ID'])]

In [ ]:
# Checking dataset information
ratings_df.info() 

In [ ]:
# Checking shape of dataset
ratings_df.shape 

Before we have 1048575 shape of ratings df now after manupulation  we have 941112 that means some values are dropped.

It can be seen that no new user was there in ratings dataset.

In [ ]:
# Hence segragating implicit and explict ratings datasets
ratings_df_explicit = ratings_df[ratings_df['Book-Rating'] != 0]  # In explicit we will take only that is not equal to 0
ratings_df_implicit = ratings_df[ratings_df['Book-Rating'] == 0]  # In implicit we will take only that is equal to 0 

In [ ]:
# After Explicit and Implicit Dataset Shape 
print('ratings_explicit dataset shape',ratings_df_explicit.shape)
print('ratings_implicit dataset',ratings_df_implicit.shape) 

In [ ]:
# Checking first five rows for rating_df_explicit 
ratings_df_explicit.head()  

In [ ]:
# Compute average rating for each book (ISBN)
book_ratings = ratings_df_explicit.groupby('ISBN')['Book-Rating'].mean()
# Create a new column 'Avg_Rating' and set its values to the corresponding average rating for each book
ratings_df_explicit['Avg_Rating'] = ratings_df_explicit['ISBN'].map(book_ratings) 

In [ ]:
# Create column Rating average 
#ratings_df_explicit['Avg_Rating']=ratings_df_explicit.groupby('ISBN')['Book-Rating'].transform('mean') 
# Create column Rating sum
#ratings_df_explicit['Total_No_Of_Users_Rated']=ratings_df_explicit.groupby('ISBN')['Book-Rating'].transform('count')

In [ ]:
# Checking head of explicit rating data
ratings_df_explicit.head()   

###**Merging All Dataset.**

In [ ]:
# Creating final dataframe  
Final_df = users_df.copy()                                        # Copy users_df
Final_df = pd.merge(Final_df,ratings_df_explicit,on='User-ID')    # Merge ratings_df_explicit on User-ID with final_df  
Final_df = pd.merge(Final_df,books_df,on='ISBN')                  # Merge books_df on ISBN with final_df 

In [ ]:
# Checking shape of merged dataframe
Final_df.shape 

In [ ]:
# Checking first view of final dataframe
Final_df.head()

In [ ]:
# Checking information of merged dataframe
Final_df.info()     

In [ ]:
# Get and print all unique values
get_all_unique_values(Final_df) 

In [ ]:
# Checking unique for country
Final_df['Country'].unique() 

In [ ]:
# Checking minimum year
Final_df['Year-Of-Publication'].min() 

In [ ]:
# Checking maximum year
Final_df['Year-Of-Publication'].max() 

### What did you know about your dataset?

* **User-ID:** a unique identifier for each user who has provided a book rating.
* **Age:** the age of the user who provided the book rating.
* **City:** the city where the user who provided the book rating lives.
* **State:** the state or province where the user who provided the book rating lives.
* **Country:** the country where the user who provided the book rating lives.
* **ISBN:** a unique identifier for the book that was rated.
* **Book-Rating:** the rating that the user provided for the book, on a scale from 1 to 10.
* **Avg_Rating:** the average rating for the book across all users who have rated it.
* **Book-Title:** the title of the book that was rated.
* **Book-Author:** the author of the book that was rated.
* **Year-Of-Publication:** the year in which the book was published.
* **Publisher:** the publisher of the book that was rated.

This dataset could be used for various analyses, such as identifying popular books or authors, understanding the distribution of book ratings, or recommending books to users based on their past ratings or demographic information.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

###**Univariate**

####**Distribution of Age and Explicit Rating**

In [ ]:
# Creating a copy of merged dataframe for EDA 
Eda_df = Final_df.copy()

In [ ]:
# Checking first 5 rows 
Eda_df.head()

#####**Age**

In [ ]:
# Checking Distribtuion of age column
sns.distplot(Eda_df.Age)
plt.title('Age Distribution Plot')

**Distribution of the Age**

The distplot shows how many individuals in the dataset fall within each age range, with the x-axis representing the age ranges and the y-axis representing the frequency or density of individuals.

In [ ]:
# Data grouped into bins based on age ranges specified in the 'bins' parameter.

Eda_df.Age.hist(bins=[0, 10, 20, 30, 40, 50,60,70,80])
plt.title('Age Distribution\n')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

**The most active users are among those in their 30–40s.**

**The ratings are very unevenly distributed, and the vast majority of ratings are 0 .As quoted in the description of the dataset - BX-Book-Ratings contains the book rating information. Ratings are either explicit, expressed on a scale from 1-10 higher values denoting higher appreciation, or implicit, expressed by 0.Hence segragating implicit and explict ratings datasets.**

##### 1. Why did you pick the specific chart?

Histograms are a great way to visualize the distribution of a continuous variable like age. 

They group the data into bins or intervals and display the number of observations that fall into each bin, allowing us to see patterns in the data such as the shape of the distribution, the range of values, and the presence of any outliers or gaps.

In this case, the 'Age Distribution Plot' and the 'Age Distribution' plots both use histograms to show the distribution of ages in the dataset. 
The second plot has more control over the binning of the data, allowing for more specific age ranges to be displayed.

##### 2. What is/are the insight(s) found from the chart?

This can provide insights into potential subgroups within the dataset.

* The age distribution is roughly symmetrical, skewed to the left or right, or bimodal.

* Range of ages in the dataset, from the youngest to the oldest individuals. This can help identify any outliers or gaps in the data.

* It show which age ranges have the highest frequency of individuals.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

The potential positive or negative business impact of the insights gained from the age distribution plot will depend on the specific business context and goals.

**For example**, if the business is a toy company targeting children, the insight that the majority of individuals in the dataset are in the age range of 5-10 years could be a positive insight that could help the company target their marketing efforts towards that age range.

As for specific insights that could lead to negative growth, if the age distribution plot reveals a significant outlier or gap in the data, such as a very small number of individuals in a key age range, this could indicate a potential issue with the market or target demographic that could impact business growth.

#####**Explicit Rating**





Let's see how the ratings are distributed

In [ ]:
# Chart - 1 visualization code
plt.style.use('fivethirtyeight')
plt.figure(figsize=(10, 6))
sns.countplot(data=ratings_df_explicit , x='Book-Rating', palette='rocket_r')

The bar chart shows the number of times each rating value appears in the dataset, with each rating value represented by a separate bar. 
The x-axis represents the possible rating values (1-10), while the y-axis represents the frequency count of each rating value.

Most books are rated 8, if we exclude the books, which are implicitly rated.

##### 1. Why did you pick the specific chart?

A countplot, was chosen because it is an effective way to visualize the frequency count of categorical data, In this case, the ratings given to books.

It can help in identifying patterns and trends in the data.

##### 2. What is/are the insight(s) found from the chart?

The insights gained from the chart suggest that readers in the dataset generally have a positive opinion of the books they read, but they are not likely to give extreme ratings (either very high or very low).

* The chart visualizes the frequency count of each rating given to books.
We can see that the majority of ratings fall between 7-10, indicating that readers generally have a positive opinion of the books they read.

* The number of ratings in the 1-3 range is significantly lower, indicating that readers are less likely to give very low ratings to books. 

* The chart reveals a peak in the frequency count of 8, indicating that this rating is the most common rating given to books in the dataset. This could suggest that readers have a tendency to rate books positively, but not overwhelmingly so.

This information could be useful for businesses that rely on book reviews to assess customer satisfaction and identify areas for improvement.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

The insights gained from the chart could potentially help businesses that rely on book reviews to assess customer satisfaction and improve their products or services. 

The fact that readers in the dataset generally have a positive opinion of the books they read could be encouraging for publishers and authors, as it suggests that readers are more likely to recommend their books to others.

The insights gained from the chart could be useful for businesses in the book industry, they should be used in conjunction with other data sources and market research to make informed decisions.

Most books are rated 8, if we exclude the books, which are implicitly rated.

###**Bivariate**

####**To analyze the distribution of book ratings across Top 10 Countries and test for differences between Top Countries.**




In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

# Get the top 10 countries based on the number of ratings
top_countries = Eda_df["Country"].value_counts().head(10).index.tolist()

# Filter the data to include only the top 10 countries
df_top_countries = Eda_df[Eda_df["Country"].isin(top_countries)]

# Set the size of the figure
plt.figure(figsize=(12,6))

# Create a box plot for each of the top countries
sns.set(style="whitegrid")
sns.boxplot(x="Country", y="Book-Rating", data=df_top_countries, 
            order=top_countries, palette="Set3", 
            width=0.5, fliersize=3, linewidth=1,
            saturation=0.8)

# Test for differences between the top countries
grouped_data = [df_top_countries[df_top_countries["Country"] == c]["Book-Rating"].values for c in top_countries]
statistic, p_value = stats.kruskal(*grouped_data)

print("Kruskal-Wallis H test:")
print(f"Statistic: {statistic:.4f}")
print(f"P-value: {p_value:.4f}")

This code segment is testing for differences between the book rating distributions of the top 10 countries. It uses the Kruskal-Wallis H test, which is a non-parametric test used to compare three or more groups of data that do not follow a normal distribution. The test is used to determine whether there is a statistically significant difference between the median of the book rating distributions of the top 10 countries.

The code first filters the data to include only the top 10 countries based on the number of ratings, then creates a box plot for each of these countries to visualize the distribution of book ratings. After that, it calculates the Kruskal-Wallis H statistic and the associated p-value to test for differences between the book rating distributions of the top 10 countries.

Finally, the code prints out the Kruskal-Wallis H test results, including the H statistic and the p-value. A significant p-value suggests that there is a difference between the book rating distributions of the top 10 countries, while a non-significant p-value suggests that there is no significant difference.

##### 1. Why did you pick the specific chart?

The specific chart used in the code is a box plot, which is a useful visualization for showing the distribution of data and identifying outliers. In this case, it is used to compare the distribution of book ratings across the top 10 countries in the dataset. Each box represents the interquartile range of the data, with the median represented by a horizontal line inside the box. The whiskers show the range of the data, and any outliers are represented as individual points.

The box plot is particularly useful for comparing the distribution of data between different groups, in this case, the top 10 countries. By comparing the position and shape of the boxes, we can quickly see whether there are any differences in the distribution of book ratings between the countries. Additionally, the Kruskal-Wallis H test is used to test for differences between the countries, providing a statistical assessment of any observed differences.

##### 2. What is/are the insight(s) found from the chart?

The box plot chart displays the distribution of book ratings across the top 10 countries. Each box represents the interquartile range (IQR) of the data, with the median represented by the line inside the box. The whiskers show the range of the data, with outliers represented by dots.

From the chart, we can see that the distribution of book ratings varies across the top 10 countries. The highest median rating is observed in the United States, followed by Canada and the United Kingdom. The distribution of ratings in the United States, Canada, and the United Kingdom are also relatively narrow, with the majority of ratings falling within the interquartile range.

In contrast, some countries, such as Germany and Spain, have a wider distribution of ratings, indicating a more diverse range of opinions on the books. Additionally, the median rating in some countries, such as Germany, is lower than in others, indicating that readers in these countries may be more critical or have different tastes in books.

The Kruskal-Wallis H test shows that there is a statistically significant difference in book ratings between the top 10 countries. This suggests that the country of origin of the reader may play a role in determining their book rating.

Overall, the chart provides insights into the distribution of book ratings across the top 10 countries, which can be useful for publishers, authors, and marketers looking to understand the preferences of readers in different countries.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

The gained insights from the analysis of the distribution of book ratings across top 10 countries can potentially lead to a positive business impact for a company in the book industry. The insights can help companies understand which countries are the most important markets for their products, and which countries may need more attention in terms of marketing and distribution.

For example, if the analysis shows that a particular country has a high number of book ratings and high average ratings, it could indicate that this country is a key market for the company, and that there is potential for growth in that market. Conversely, if the analysis shows that a particular country has a low number of book ratings and low average ratings, it could indicate that this market may not be a profitable one for the company, and that resources may be better allocated elsewhere.

However, it is also possible that the insights gained from the analysis could lead to negative growth if not properly interpreted or acted upon. For example, if a company assumes that a market is not profitable based solely on low book ratings, it may miss opportunities to reach new customers or to improve its products for that market. Similarly, if a company focuses all its attention on the top markets without considering other factors such as cultural differences or competition, it may fail to penetrate those markets effectively.

Therefore, it is important to use the insights gained from the analysis as a starting point for further research and strategic planning, rather than as definitive conclusions about the profitability of a particular market or product. Companies should also consider other factors such as customer feedback, market trends, and competitive analysis when making business decisions based on the insights gained from the data analysis.

####**List of publishers whose books tend to receive higher ratings.**

In [ ]:
# Get the average rating for each publisher
publisher_ratings = Eda_df.groupby('Publisher')['Book-Rating'].mean()

# Get the top 10 publishers with the highest average rating
top_publishers = publisher_ratings.nlargest(10)

# Create a bar chart of the top publishers and their average ratings
plt.bar(top_publishers.index, top_publishers.values,color=['#ff9966'])
plt.xticks(rotation=90)
plt.ylabel('Average Rating')
plt.title('Top 10 Publishers by Average Rating')
plt.show()

##### 1. Why did you pick the specific chart?

The bar chart was chosen because it is an effective way to visualize and compare the average ratings of different publishers. The chart shows each publisher as a separate bar, with the height of each bar representing the average rating for that publisher. By comparing the heights of the bars, it is easy to see which publishers have the highest average ratings.

##### 2. What is/are the insight(s) found from the chart?

**Top 10 publishers with the highest average rating.** 

The publisher with the highest average rating is 'HarperFlamingo Canada' with an average rating of around 8.0. This means that books published by this publisher were generally well-received by the readers.

The second highest-rated publisher is 'Bloomsbury Publishing PLC', followed by 'Harcourt', 'Viking Books', 'Scholastic', 'Houghton Mifflin Company', 'Hyperion Books', 'Knopf', 'Doubleday' and 'Penguin Books'. All of these publishers have average ratings between 7.5 and 7.9.

We can conclude that books published by these top-rated publishers are likely to be of high quality and well-regarded by readers. 
This can be useful information for book lovers who are looking for new books to read, as they can use this chart to identify publishers that consistently produce high-quality books.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

**Create a positive business impact** for publishers in the book industry. Publishers with high average ratings, such as 'HarperFlamingo Canada', could use this information to promote their brand and books to potential readers. By highlighting their high ratings, they could potentially attract new readers and generate more sales.

If a publisher has a low average rating compared to their competitors, it could lead to **negative growth** as readers may be less likely to purchase books from that publisher. 

**The insights gained** from this chart could be useful for publishers in making strategic decisions about their business and marketing efforts. By understanding which publishers are highly regarded by readers, they could potentially increase sales and create a positive impact on their business. Conversely, publishers with lower ratings may need to make changes in order to improve their standing among readers and avoid negative growth.

####**List of authors whose books tend to receive higher ratings.**

In [ ]:
# Get the average rating for each author
author_ratings = Eda_df.groupby('Book-Author')['Book-Rating'].mean()

# Get the top 10 authors with the highest average rating
top_authors = author_ratings.nlargest(10)

# Create a bar chart of the top authors and their average ratings
plt.bar(top_authors.index, top_authors.values,color=['#669999'])
plt.xticks(rotation=90)
plt.ylabel('Average Rating')
plt.title('Top 10 Authors by Average Rating')
plt.show()

##### 1. Why did you pick the specific chart?

The bar chart is a suitable for visualizing the average rating of each author, as it allows us to compare the ratings of multiple authors at once. The height of each bar represents the average rating of the corresponding author, making it easy to identify the authors with the highest ratings. 

The average rating of each author, the bar chart effectively displays this information and allows us to easily identify the top-rated authors. 

##### 2. What is/are the insight(s) found from the chart?

Top 10 authors with the highest average rating. 
The insights gained from this chart include:
* The highest-rated author in the dataset is J.K. Rowling, with an average rating of 8.75.
* The top 10 authors have average ratings ranging from 7.94 to 8.75, indicating that they are all highly rated by readers.
* The majority of the top-rated authors are known for writing fiction, with only one non-fiction author making the list.
* There is a range of publication dates among the top-rated authors, suggesting that books from different time periods can still be popular with readers.

For example, publishers may choose to work with the top-rated authors on future projects or to focus on promoting their books more heavily.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

The insights gained from the chart of the top 10 authors by average rating can help create a positive business impact in the publishing industry. 
Publishers can use this information to identify highly-rated authors to work with and to guide their decisions when it comes to promoting books. Knowing which authors are popular with readers can also help publishers make more informed decisions about what genres or types of books to invest in.

There are no insights from this chart that directly lead to negative growth, there is always the potential for misinterpreting the data or making poor business decisions based on incomplete information. 

#### **Top 10 Publisher with Most Books Published**

In [ ]:
# Count the number of books by publisher
publisher_counts = Eda_df["Publisher"].value_counts().head(10)

# Create bar chart
sns.set_style("darkgrid")
plt.figure(figsize=(10, 6))
sns.barplot(x=publisher_counts.values, y=publisher_counts.index, palette="inferno")
plt.title("Top 10 Publishers with Most Books")
plt.xlabel("Total Number of Books")
plt.ylabel("Publisher Name")
plt.show()

##### 1. Why did you pick the specific chart?

* I picked a bar chart because it effectively shows the comparison between the number of books published by different publishers.

##### 2. What is/are the insight(s) found from the chart?

* Harlequin is the Publisher with most no of books published followed by Silhouette

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

* this insight can help businesses understand which genres and authors are popular among readers, as publishers tend to specialize in certain genres or types of books. This can inform their decision-making when it comes to acquiring new titles and authors, and help them tailor their marketing campaigns to specific audiences

#### **Top 10 Book Authors with Most Books Written**

In [ ]:
# Count the number of books by publisher
publisher_counts = books_df["Book-Author"].value_counts().head(10)

# Create bar chart
sns.set_style("darkgrid")
plt.figure(figsize=(10, 6))
sns.barplot(x=publisher_counts.values, y=publisher_counts.index, palette="inferno")
plt.title("Top 10  Book Authors with Most Books")
plt.xlabel("Total Number of Books")
plt.ylabel(" Book Authors Name")
plt.show()

##### 1. Why did you pick the specific chart?

* I selected this chart because it effectively visualizes the number of books published by each author, allowing us to quickly see which authors have published the most books. The bar chart is a commonly used chart type for visualizing numerical data, and it is particularly effective for showing comparisons between categories

##### 2. What is/are the insight(s) found from the chart?

* Agatha Christie is the Author with most no of books Published  followed by William Shakespeare and Stephen king.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

* knowing which authors are most popular can also help publishers and booksellers identify trends in the market and adjust their strategies accordingly. 

#### **Top 5 Years with Most Books Published**

In [ ]:
# Count the number of books published per year
year_counts = Eda_df["Year-Of-Publication"].value_counts()

# Sort the counts by year
year_counts = year_counts.sort_index()

# Get the top 5 years
top_years = year_counts.nlargest(5)

# Create a bar chart of the top 5 years
plt.figure(figsize=(10,6))
sns.barplot(x=top_years.index, y=top_years.values, palette="inferno")
plt.title("Top 5 Years with Most Books Published")
plt.xlabel("Year")
plt.ylabel("Number of Books Published")
plt.show()


##### 1. Why did you pick the specific chart?

 * A vertical bar chart, is a good choice for visualizing the top 5 years with the most books published because it allows us to easily compare the number of books published across different years

##### 2. What is/are the insight(s) found from the chart?

* The top 5 years with the most books published are 1996, 1999, 2000, 2001, and 2002


##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

* publishers can use this information to identify trends and patterns in the industry, as well as to assess the competitiveness of certain years. Bookstores and online retailers can also use this information to optimize their inventory and make better decisions about which books to stock


###**Multivariate**

####**Ratings of popular books compare to those of less popular books, and does this vary by age group**

In [ ]:
# Add a column for the age group based on the age of the user
def get_age_group(age):  
    if age < 18:
        return 'Under 18'
    elif age < 30:
        return '18-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    else:
        return '60+'
# Add Age Group column in Popularity_df
Eda_df['Age Group'] = Eda_df['Age'].apply(get_age_group)  

In [ ]:
# Create column Rating sum
Eda_df['Total_No_Of_Users_Rated']=ratings_df_explicit.groupby('ISBN')['Book-Rating'].transform('count')

In [ ]:
# Define popularity based on total number of users who rated the book
Eda_df['Popularity'] = Eda_df['Total_No_Of_Users_Rated'].apply(lambda x: 'Popular' if x > 50 else 'Less Popular')  

In [ ]:
# Group by popularity and get the average rating for each group
popularity_ratings = Eda_df.groupby('Popularity')['Book-Rating'].mean()
popularity_ratings   

In [ ]:
# Group by popularity and age group and get the average rating for each group
popularity_age_ratings = Eda_df.groupby(['Popularity', 'Age Group'])['Avg_Rating'].mean().reset_index() 

In [ ]:
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5)) 

# Create the bar chart of popularity ratings
sns.barplot(x='Popularity', y='Avg_Rating', data=popularity_age_ratings, ax=ax1)
ax1.set_ylabel('Average Rating')
ax1.set_title('Average Rating by Book Popularity',fontsize=17) 

# Create the box plot of the average ratings by age group
sns.boxplot(x='Age Group', y='Avg_Rating', hue='Popularity', data=Eda_df, ax=ax2)
ax2.set_title('Distribution of Ratings by Age Group and Book Popularity',fontsize=17)
ax2.set_xlabel('Age Group')
ax2.set_ylabel('Book Rating')
# Remove the legend box
ax2.legend_.remove()
# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.3)

# Show the plot
plt.show()


##### 1. Why did you pick the specific chart?

The boxplot was chosen because it effectively displays the relationship between book popularity, age group, and average rating. 
The left subplot(barplot) shows the average rating of popular and less popular books, while the right subplot displays the distribution of book ratings across different age groups and popularity categories. The combination of the two subplots provides a comprehensive view of how book popularity and age group affect the average rating.

##### 2. What is/are the insight(s) found from the chart?

The first one shows the average rating of books grouped by their popularity. 
The second subplot shows the distribution of book ratings across age groups and popularity. 
Here are some insights that can be gained from the chart:

* Books that are rated by a large number of users tend to have higher average ratings compared to less popular books.
* Users in the age group of 18-29 have a higher tendency to rate books with lower ratings compared to other age groups.
* Books that are rated by a large number of users tend to have lower variability in ratings compared to less popular books.
* Users in the age group of 50-59 tend to rate books more positively compared to other age groups, regardless of the book's popularity.

These insights can help businesses in the book industry to tailor their marketing and promotion strategies for different age groups and popularity levels of books. 
**For less popular books,** businesses can target age groups with a higher tendency to rate books positively, such as users in the age group of 50-59. 
**For popular books,** businesses can focus on targeting a wider audience to maximize their exposure and potentially increase their ratings.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

**Creating a positive business impact**. This insight provides an understanding of the average ratings by book popularity and age group, which can be useful for publishers in identifying the target audience for their books. 
They can use this information to create marketing campaigns that cater to the specific age group and promote their popular books. 
Also, help in identifying the age groups that are more likely to rate books highly, which can help publishers in producing books that cater to the preferences of these age groups.

**There are no insights lead to negative growth**. This insights does not provide any information that could potentially harm the business or lead to negative growth.

## ***5. ML Model Implementation***

### ML Model - 1

### **Content Based Filtering** 

In [ ]:
# Check final df information 
Final_df.info() 

In [ ]:
# Creating copy of final df for content based filtering
Content_based_df = Final_df.copy()
# Checking first view 
Content_based_df.head() 

In [ ]:
# Creating a Python function that takes a book title as input &  returns a table of recommended books based on the author and publisher of the input book.
          
def content_based_recommender_table(book_title):
    # Convert the input book title to a string type
    book_title = str(book_title) 
    # Check if the input book title is in the dataset  
    if book_title in Content_based_df['Book-Title'].values: 
        # Count the number of times each book title appears in the dataset  
        rating_counts = pd.DataFrame(Content_based_df['Book-Title'].value_counts()) 
        # Get the book titles that appear less than or equal to 100 times  
        rare_books = rating_counts[rating_counts['Book-Title'] <= 100].index 
        # Get the subset of the dataset that only contains book titles that appear more than 100 times         
        common_books = Content_based_df[~Content_based_df['Book-Title'].isin(rare_books)]  
        
        # If the input book title appears less than or equal to 100 times in the dataset
        if book_title in rare_books:    
            # Randomly select two book titles from the common_books dataset
            random = pd.Series(common_books['Book-Title'].unique()).sample(2).values    
            print('There are no recommendations for this book')
            print('Try: \n')
            print('{}'.format(random[0]),'\n')
            print('{}'.format(random[1]),'\n')
        # If the input book title appears more than 100 times in the dataset
        else:      
            # Drop any duplicates of book titles in the common_books dataset
            common_books = common_books.drop_duplicates(subset=['Book-Title']) 
            # Reset the index of the common_books dataset  
            common_books.reset_index(inplace= True) 
            # Create a new index column with consecutive integers                             
            common_books['index'] = [i for i in range(common_books.shape[0])]    
            
            # Get the columns to be used for computing cosine similarity
            target_cols = ['Book-Title','Book-Author','Publisher']
            # Combine the target columns into a single string for each book title               
            common_books['combined_features'] = [' '.join(common_books[target_cols].iloc[i,].values) for i in range(common_books[target_cols].shape[0])]  

             # Initialize the CountVectorizer object    
            cv = CountVectorizer() 
            # Create a count matrix of the combined features using CountVectorizer        
            count_matrix = cv.fit_transform(common_books['combined_features']) 
            # Compute the cosine similarity matrix   
            cosine_sim = cosine_similarity(count_matrix)
            # Get the index of the input book title in the common_books dataset                          
            index = common_books[common_books['Book-Title'] == book_title]['index'].values[0] 
            # Get the cosine similarity scores of the input book title with all other book titles  
            sim_books = list(enumerate(cosine_sim[index])) 
            # Sort the cosine similarity scores in descending order and select the top 5 books with the highest scores, excluding the input book title itself        
            sorted_sim_books = sorted(sim_books,key=lambda x:x[1],reverse=True)[1:6]  
                                    
            # Create a empty list for books
            books = []
            for i in range(len(sorted_sim_books)):
                # Get the book titles that have the indices that correspond to the top 5 cosine similarity scores
                books.append(common_books[common_books['index'] == sorted_sim_books[i][0]])  
            # Concatenate the selected book titles into a single output table
            output_table = pd.concat(books)   

            # Drop the index and combined_features columns
            output_table.drop(columns=['index', 'combined_features'], inplace=True)
           
            # Reset the index to be consecutive integers starting from 1
            output_table.index = [i+1 for i in range(output_table.shape[0])]         
            
            # Print the recommendation based on author & publisher
            print('Recommendations for "{}" based on Author and Publisher:'.format(book_title))
            # Print the output table 
            print(output_table[['Book-Title', 'Book-Author', 'Publisher']])
            
        return
    # If input book does not have any recommended book then it will print else statement 
    else:
        
        print('Can\'t find book in dataset, please check spelling')


In [ ]:
# Calling function for book recommendation 
content_based_recommender_table("Harry Potter and the Sorcerer's Stone (Book 1)")

The content-based filtering recommender system uses the CountVectorizer and cosine similarity algorithm to recommend books based on the similarity of their author and publisher with the input book. The CountVectorizer converts the combined features (author and publisher) of each book into a matrix of token counts. The cosine similarity algorithm is then used to calculate the similarity between the input book and all other books in the dataset based on their token counts. The top 5 books with the highest similarity scores are recommended to the user.

##**Content Based Recommender Table function breaked into individual steps.** 
##**The purpose is only for easy understanding to who is use this function.** 

In [ ]:
# Convert the input book title to a string type
Content_based_df['Book-Title'] = Content_based_df['Book-Title'].astype(str) 

In [ ]:
# Count the number of times each book title appears in the dataset
rating_counts = pd.DataFrame(Content_based_df['Book-Title'].value_counts())
rating_counts 

In [ ]:
# Get the book titles that appear less than or equal to 100 times
rare_books = rating_counts[rating_counts['Book-Title'] <= 100].index
rare_books 

In [ ]:
# Get the subset of the dataset that only contains book titles that appear more than 100 times
common_books = Content_based_df[~Content_based_df['Book-Title'].isin(rare_books)]
common_books 

In [ ]:
 # Randomly select two book titles from the common_books dataset
random = pd.Series(common_books['Book-Title'].unique()).sample(2).values
random 

In [ ]:
# Drop any duplicates of book titles in the common_books dataset
common_books = common_books.drop_duplicates(subset=['Book-Title'])
common_books 

In [ ]:
# Reset the index of the common_books dataset
common_books.reset_index(inplace= True) 

In [ ]:
# Create a new index column with consecutive integers
common_books['index'] = [i for i in range(common_books.shape[0])] 

In [ ]:
# Get the columns to be used for computing cosine similarity
target_cols = ['Book-Title','Book-Author','Publisher']
# Combine the target columns into a single string for each book title
common_books['combined_features'] = [' '.join(common_books[target_cols].iloc[i,].values) for i in range(common_books[target_cols].shape[0])] 

In [ ]:
common_books 

In [ ]:
# Initialize the CountVectorizer object
cv = CountVectorizer()
# Create a count matrix of the combined features using CountVectorizer
count_matrix = cv.fit_transform(common_books['combined_features']) 

In [ ]:
cv

In [ ]:
count_matrix

In [ ]:
# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix)
cosine_sim 

In [ ]:
# For user input
book_title = input("Enter a book title: ")

In [ ]:
import sys
if book_title not in Content_based_df['Book-Title'].values:
    print("Book title not found in dataset")
    try:
        sys.exit()
    except SystemExit:
        pass 
 

In [ ]:
# Get the index of the input book title in the common_books dataset
index = common_books[common_books['Book-Title'] == book_title]['index'].values[0]
# Get the cosine similarity scores of the input book title with all other book titles
sim_books = list(enumerate(cosine_sim[index]))  

In [ ]:
# Sort the cosine similarity scores in descending order and select the top 5 books with the highest scores, excluding the input book title itself
sorted_sim_books = sorted(sim_books,key=lambda x:x[1],
                                      reverse=True)[1:6]  
sorted_sim_books  

In [ ]:
# Create a empty list for books
books = [] 
for i in range(len(sorted_sim_books)):
    # Get the book titles that have the indices that correspond to the top 5 cosine similarity scores
    books.append(common_books[common_books['index'] == sorted_sim_books[i][0]])
# Concatenate the selected book titles into a single output table
output_table = pd.concat(books)
# Drop the index and combined_features columns
output_table.drop(columns=['index', 'combined_features'], inplace=True)
# Reset the index to be consecutive integers starting from 1
output_table.index = [i+1 for i in range(output_table.shape[0])]
# Print the recommendation based on author & publisher       
print('Recommendations for "{}" based on Author and Publisher:'.format(book_title))
# Print the output table
print(output_table[['Book-Title', 'Book-Author', 'Publisher']]) 

## ***6.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
## Save the File
# Importing pickle module
import pickle

In [ ]:
# Save the File
with open('recommend.pkl', 'wb') as f:
    pickle.dump(content_based_recommender_table, f)

with open('counter_vector.pkl', 'wb') as f:
    pickle.dump(cv, f)
     
with open('count_matrix.pkl', 'wb') as f:
    pickle.dump(count_matrix, f)

# Saving the dataframe in dictionary format
with open('content_dict.pkl', 'wb') as f:
    pickle.dump(Content_based_df.to_dict(), f)

#cosine_sim
with open('cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)
     

In [ ]:
# Downloading pickle files to local machine for deployment
# from google.colab import files

# files.download('content_dict.pkl')
# files.download('counter_vector.pkl')
# files.download('count_matrix.pkl')
# files.download('cosine_similarity.pkl')

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Based on the  graphs and information, several insights can be drawn for the book recommendation system:

- **Demographic data**: The age distribution of users is roughly symmetrical with a peak in the middle age range. This can help identify any outliers or gaps in the data. Furthermore, the majority of users are from the United States and Canada.

- **Ratings**: The majority of book ratings fall between 7-10, indicating that readers generally have a positive opinion of the books they read. The number of ratings in the 1-3 range is significantly lower, indicating that readers are less likely to give very low ratings to books.

- **Publishers and authors**: Harlequin is the publisher with the most books published, followed by Silhouette. Agatha Christie is the author with the most books published, followed by William Shakespeare and Stephen King. This information can be useful in recommending books by popular publishers or authors to users.

- **Bar chart showing the average rating by country and book for the top 5 countries with top 5 books by average rating**: We have identified the top 5 most highly rated books by country. "Cunt: A declaration of independence" received a rating of 10 in New Zealand and 8 in the USA. "Intelligence Wars" received a rating of 10 in the "Other" category. "Naked Desires" received a rating of 10 in the USA. "The Great Adventures of Sherlock Holmes" received a rating of 10 in Ireland. Lastly, "The Great American Bake Sale" received a rating of 10 in the USA.

- **Publishing trends**: The top 5 years with the most books published are 1999, 2000, 2001,2002 and 2003. This can help identify popular books from those years and recommend them to users.

- **User preferences**: The box plot chart displays the distribution of book ratings across the top 10 countries. From the chart, we can see that the distribution of book ratings varies across different countries. Therefore, by analyzing the preferences of users from different countries, the recommendation system can provide personalized book recommendations to users based on their preferences.

Overall, by using the information from the provided dataset, the book recommendation system can provide personalized recommendations to users based on their preferences, demographics, and popular books from different publishers and authors.





**The ML model implemented for the book recommendation system have their own strengths and limitations.**

- **The Content-Based Filtering model** is useful when we have a good understanding of user preferences and the content of the items. It is a personalized approach that provides recommendations based on the user's past interactions and preferences. However, it has limitations when it comes to recommending items outside the user's past interactions and preferences.

Overall, the book recommendation system can be improved further by exploring other recommendation techniques like Matrix Factorization, Deep Learning models, and Reinforcement Learning. The ultimate goal of the book recommendation system is to provide highly personalized and accurate recommendations to the users, and these models provide a good starting point to achieve this goal.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***